In [74]:
import subprocess
import csv

In [75]:
def ports_convert_to_int(port):
    try:
        return int(port)
    except:
        return port

def load_table_ports(filename):
    """Load ports table, that contain ports registered by IANA and ICANN, from csv and return it as dictionary.

    Returns:
        dictionary: Loaded ports table as a dictionary (port->service_name).
    """
    if filename.endswith(".csv") is False:
        print("The filename of table contains services haven't suffix or isn't .csv")
        sys.exit(1)
    if os.path.isfile(filename) is False:
        print(f"The file with name {filename} doesn't exists.")
        sys.exit(1)
    try:
        with open(filename, mode="r", encoding="utf-8") as infile:
            reader = csv.reader(infile)
            reg_ports = dict(
                (ports_convert_to_int(rows[1]), rows[0]) for rows in reader
            )
        return reg_ports
    except:
        print(f"Error in loading file {filename}")
        sys.exit(1)


In [76]:
reg_ports_tb = load_table_ports("Ports.csv")

In [77]:
def check_port(port, ports_tb):
    """Check if port used in dependency is service or registered.

    Args:
        port (int): Integer of used port by device.
        services_tb (dictionary): Dictionary contains list of services.
        ports_tb (dictionary): Dictionary contains registered port defined by IANA and ICAN.

    Returns:
        bool: True if port is service or registered.
    """
    if ports_tb.get(port) is not None:
        if ports_tb.get(port) == "":
            return False
        return True
    return False

In [78]:
returned_text = subprocess.check_output("lsof -i -n -P", shell=True, universal_newlines=True)

In [79]:
tmp = returned_text.split("\n")
header = tmp[0]
connections = tmp[1:]

In [80]:
header

'COMMAND     PID   USER   FD   TYPE DEVICE SIZE/OFF NODE NAME'

In [86]:
for conn in connections:
    items = conn.split()
    if len(items) == 0 or "*" in items[-1]:
        continue
    status = items[-1]
    if status == "(LISTEN)":
        continue
    application = items[0]
    dependency = items[-2]

    direction = None
    if '->' in dependency:
        direction = False
        sides = dependency.split('->')
    elif '<-' in dependency:
        direction = True
        sides = dependency.split('<-')
    else:
        continue    
    if len(sides) != 2:
        continue
    # source side
    if '[' in sides[0]:
        # IPv6
        tmp = sides[0].split("[")[-1].split("]")
        ip_s = tmp[0]
        port_s = int(tmp[1].split(":")[-1])
    else:
        #IPv4
        tmp = sides[0].split(":")
        ip_s = tmp[0]
        port_s = int(tmp[1])
    # destination side
    if '[' in sides[1]:
        # IPv6
        tmp = sides[1].split("[")[-1].split("]")
        ip_d = tmp[0]
        port_d = int(tmp[1].split(":")[-1])
    else:
        #IPv4
        tmp = sides[1].split(":")
        ip_d = tmp[0]
        port_d = int(tmp[1])
    if ip_d == "127.0.0.1" and ip_s == "127.0.0.1":
        continue

    tmp_port_s = check_port(port_s, reg_ports_tb)
    tmp_port_d = check_port(port_d, reg_ports_tb)
    if tmp_port_s is True and tmp_port_d is True:
        if direction is False:
            id_dependency = f"{ip_d}({port_d})-{ip_s}"
        else:
            id_dependency = f"{ip_s}({port_s})-{ip_s}"
    elif tmp_port_s is True:
        id_dependency = f"{ip_s}({port_s})-{ip_d}"
    elif tmp_port_d is True:
        id_dependency = f"{ip_d}({port_d})-{ip_s}"
    else:
        id_dependency = f"{ip_s}({port_s}-{port_d})-{ip_d}"
    new_row = [application, id_dependency]
    with open('output.csv','r') as f:
        existingLines = [line for line in csv.reader(f, delimiter=',')]
        if new_row in existingLines:
            continue
    with open('output.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(new_row)

In [1]:
s = "09:37:48.817004 IP 147.32.76.118.42758 > 3.68.124.168.443: Flags [P.], seq 3734151325:3734151815, ack 1248824436, win 3631, options [nop,nop,TS val 2609698556 ecr 2924979190], length 490"


In [2]:
tmp = s.split()
ip_s_port_s = tmp[2]
ip_d_port_d = tmp[4].split(":")[0]
print(ip_s_port_s)
print(ip_d_port_d)

147.32.76.118.42758
3.68.124.168.443


In [5]:
tmp = ip_s_port_s.split(".")
port_s = tmp[-1]
ip_s = ""
for i in range(len(tmp) - 1):
    if i != len(tmp)-2:
        ip_s += f"{tmp[i]}."
    else:
        ip_s += tmp[i]
ip_s

'147.32.76.118'